<i18n value="e6dedae8-1335-494e-acdf-4a1906f8c826"/>


# Using Auto Loader and Structured Streaming with Spark SQL

## Learning Objectives
By the end of this lab, you should be able to:
* Ingest data using Auto Loader
* Aggregate streaming data
* Stream data to a Delta table

<i18n value="ab5018b7-17b9-4f66-a32d-9c86860f6f30"/>


## Setup
Run the following script to setup necessary variables and clear out past runs of this notebook. Note that re-executing this cell will allow you to start the lab over.

In [0]:
%run ../Includes/Classroom-Setup-06.3L

Python interpreter will be restarted.
Python interpreter will be restarted.


Resetting the learning environment:
| dropping the schema "jtschopp_k017_dbacademy_dewd"...(1 seconds)
| removing the working directory "dbfs:/mnt/dbacademy-users/jtschopp@u.rochester.edu/data-engineering-with-databricks"...(0 seconds)

Skipping install of existing datasets to "dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02"

Validating the locally installed datasets:
| listing local files...(8 seconds)
| completed (8 seconds total)

Creating & using the schema "jtschopp_k017_dbacademy_dewd"...(1 seconds)
Predefined tables in "jtschopp_k017_dbacademy_dewd":
| -none-

Predefined paths variables:
| DA.paths.working_dir: dbfs:/mnt/dbacademy-users/jtschopp@u.rochester.edu/data-engineering-with-databricks
| DA.paths.user_db:     dbfs:/mnt/dbacademy-users/jtschopp@u.rochester.edu/data-engineering-with-databricks/database.db
| DA.paths.datasets:    dbfs:/mnt/dbacademy-datasets/data-engineering-with-databricks/v02
| DA.paths.checkpoints: dbfs:/mnt/dbacademy-users/jtschopp@u.

<i18n value="03347519-151b-4304-8cda-1cbd91af0737"/>



## Configure Streaming Read

This lab uses a collection of customer-related CSV data from the **retail-org/customers** dataset.

Read this data using <a href="https://docs.databricks.com/spark/latest/structured-streaming/auto-loader.html" target="_blank">Auto Loader</a> using its schema inference (use **`customers_checkpoint_path`** to store the schema info). Create a streaming temporary view called **`customers_raw_temp`**.

In [0]:
# TODO
dataset_source = f"{DA.paths.datasets}/retail-org/customers/"
customers_checkpoint_path = f"{DA.paths.checkpoints}/customers"

(spark
  .readStream
  .format("cloudFiles")
  .option("cloudFiles.format", "csv")
  .option("cloudFiles.schemaLocation", customers_checkpoint_path)
  .load(dataset_source)
  .createOrReplaceTempView("customers_raw_temp"))

In [0]:
from pyspark.sql import Row
assert Row(tableName="customers_raw_temp", isTemporary=True) in spark.sql("show tables").select("tableName", "isTemporary").collect(), "Table not present or not temporary"
assert spark.table("customers_raw_temp").dtypes ==  [('customer_id', 'string'),
 ('tax_id', 'string'),
 ('tax_code', 'string'),
 ('customer_name', 'string'),
 ('state', 'string'),
 ('city', 'string'),
 ('postcode', 'string'),
 ('street', 'string'),
 ('number', 'string'),
 ('unit', 'string'),
 ('region', 'string'),
 ('district', 'string'),
 ('lon', 'string'),
 ('lat', 'string'),
 ('ship_to_address', 'string'),
 ('valid_from', 'string'),
 ('valid_to', 'string'),
 ('units_purchased', 'string'),
 ('loyalty_segment', 'string'),
 ('_rescued_data', 'string')], "Incorrect Schema"

<i18n value="4582665f-8192-4751-83f8-8ae1a4d55f22"/>



## Define a streaming aggregation

Using CTAS syntax, define a new streaming view called **`customer_count_by_state_temp`** that counts the number of customers per **`state`**, in a field called **`customer_count`**.

In [0]:
%sql
-- TODO

CREATE OR REPLACE TEMPORARY VIEW customer_count_by_state_temp AS
SELECT
  state,
  count(customer_id) AS customer_count
  FROM customers_raw_temp
  GROUP BY
  state

In [0]:
assert Row(tableName="customer_count_by_state_temp", isTemporary=True) in spark.sql("show tables").select("tableName", "isTemporary").collect(), "Table not present or not temporary"
assert spark.table("customer_count_by_state_temp").dtypes == [('state', 'string'), ('customer_count', 'bigint')], "Incorrect Schema"

<i18n value="bef919d7-d681-4233-8da5-39ca94c49a8b"/>



## Write aggregated data to a Delta table

Stream data from the **`customer_count_by_state_temp`** view to a Delta table called **`customer_count_by_state`**.

In [0]:
# TODO
customers_count_checkpoint_path = f"{DA.paths.checkpoints}/customers_count"

query = (spark.table("customer_count_by_state_temp")
              .writeStream
              .format("delta")
              .option("checkpointLocation", customers_count_checkpoint_path)
              .outputMode("complete")
              .table("customer_count_by_state"))

In [0]:
DA.block_until_stream_is_ready(query)

Processed 0 of 2 batches...
Processed 1 of 2 batches...
Processed 1 of 2 batches...
Processed 1 of 2 batches...
Processed 1 of 2 batches...
Processed 2 of 2 batches...
The stream is now active with 2 batches having been processed.


In [0]:
assert Row(tableName="customer_count_by_state", isTemporary=False) in spark.sql("show tables").select("tableName", "isTemporary").collect(), "Table not present or not temporary"
assert spark.table("customer_count_by_state").dtypes == [('state', 'string'), ('customer_count', 'bigint')], "Incorrect Schema"

<i18n value="f74f262f-10c4-4f2f-84d6-f69e56c54ac6"/>



## Query the results

Query the **`customer_count_by_state`** table (this will not be a streaming query). Plot the results as a bar graph and also using the map plot.

In [0]:
%sql
-- TODO
SELECT * FROM customer_count_by_state

state,customer_count
MT,203
PA,930
NH,2
AK,37
WI,992
OH,1914
WA,669
UT,416
ND,61
ME,339


<i18n value="e2cf644d-96f9-47f7-ad81-780125d3ad4b"/>


## Wrapping Up

Run the following cell to remove the database and all data associated with this lab.

In [0]:
DA.cleanup()

Resetting the learning environment:
| stopping the stream "None"...(0 seconds)
| dropping the schema "jtschopp_k017_dbacademy_dewd"...(3 seconds)
| removing the working directory "dbfs:/mnt/dbacademy-users/jtschopp@u.rochester.edu/data-engineering-with-databricks"...(0 seconds)

Validating the locally installed datasets:
| listing local files...(8 seconds)
| completed (8 seconds total)



<i18n value="8f3c4c52-b5d9-4f8a-974c-ce5db6430c43"/>


By completing this lab, you should now feel comfortable:
* Using PySpark to configure Auto Loader for incremental data ingestion
* Using Spark SQL to aggregate streaming data
* Streaming data to a Delta table